### Install dependencies

In [ ]:
!pip install gizeh pydub
!pip install moviepy --upgrade
!pip install ffmpeg --upgrade


### Get audio samples

In [ ]:
# French great podcast 
!wget https://audio.audiomeans.fr/file/eiYNOubrDD/65beb744-01fc-40e2-8bb0-312c16855cbb.mp3 
!mv *.mp3 ../assets/

In [ ]:
# English podcast
!gsutil cp gs://github-repo/generative-ai/gemini/long-context/*.mp3 ../assets/

### Change the quality of the he audio file

In [37]:
from pydub.utils import mediainfo
from pydub import AudioSegment

def change_bitrate(file    ,root_dir = "../assets/", output_dir = "../assets/resampling/"):
    media_info = mediainfo(root_dir+file)
    print(media_info)
    original_bitrate = media_info['bit_rate']

    print(f"original_bitrate = {original_bitrate}") 
    bitrates = [#original_bitrate]# 
        "64k", "128k", "256k"
        #, 312000
        ]

    frame_rates = [8000, 16000, 22000, 44000, 48000]

    frame_rate = ""
    # Iterate over each audio files
    for bitrate in bitrates:
        for frame_rate in frame_rates:
            sound = AudioSegment.from_mp3(root_dir+file,
                                        # 2 byte (16 bit) samples
                                        #sample_width=1,
                                        # frame rate
                                        frame_rate=frame_rate,
                                        # Mono
                                        channels=1
                                        )
            

            print(80*"__")
            print(f"file = {file}")

            print(f"duration_seconds = {sound.duration_seconds}")
            print(f"sample_width = {sound.sample_width}")
            print(f"channels = {sound.channels}")
            print(f"frame_rate = {sound.frame_rate}")
            sound = sound.set_frame_rate(frame_rate)
            print(f"frame_rate updated = {sound.frame_rate}")

            sound.export(output_dir+f"{file}-{bitrate}-{frame_rate}.mp3" , format="mp3"
                        , bitrate=bitrate
                        )
        



In [ ]:
file_name = "65beb744-01fc-40e2-8bb0-312c16855cbb.mp3"
change_bitrate(file_name)

## Change the format ==> Convert to video

In [ ]:
from audioedit import combine_audio_into_video
from moviepy.editor import VideoFileClip, TextClip, ColorClip
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
from pydub.utils import mediainfo

def convert_to_wav_16k(audio_path):
    output_filename = audio_path.replace(".mp3", ".wav") 
    AudioSegment.from_mp3(audio_path).export(output_filename, format="wav", parameters=["-ar", "16000"])

    return output_filename

def convert_to_video(audio_path):
    output_filename = audio_path.replace(".mp3", ".mp4") 
    #output_filename = audio_path + ".mp4"
    # original_bitrate = mediainfo(audio_path)['bit_rate']
    # print("original_bitrate:",original_bitrate)

    # print(mediainfo(audio_path))
    
    # Load audio clips
    audio_clips = AudioFileClip(audio_path)
    total_audio = audio_clips.duration 
    print(f"total_audio = {total_audio}")
    video_clip = ColorClip(size=(1,1), color=(255,255,255) #, duration=total_audio
                           )

    # Combine and save the video
    video_clip = video_clip.set_audio(audio_clips).set_duration(total_audio)
    
    #video_clip.set_duration(total_audio)
    video_clip.write_videofile(
        filename=output_filename, fps=1, codec="libx264",
                        bitrate="1k", 
                        audio=True, audio_fps=16000,
                        preset="medium",
                        audio_nbytes=4, audio_codec="libmp3lame",
                        audio_bitrate="128k", audio_bufsize=2000,
                        #temp_audiofile=None,
                        rewrite_audio=False, remove_temp=True,
                        write_logfile=False, verbose=True,
                        #threads=None, 
                        #ffmpeg_params=None
    )

    return output_filename
        # output_filename, #codec="libx264",
        #                                                   audio = True, fps=1)

file_name = "/Users/julienmiquel/dev/STT/assets/Interview-Stephane-Gachet-JO.mp3"


#convert_to_video(file_name)    
convert_to_wav_16k(file_name)

In [ ]:
gcs.write_file_to_gcs(config.BUCKET,f"stt/Interview-Stephane-Gachet-JO.mp4","/Users/julienmiquel/dev/STT/assets/Interview-Stephane-Gachet-JO.mp4")



In [ ]:
import glob
from pydub import AudioSegment


root_dir = "/Users/julienmiquel/dev/STT/stt/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                    "*.mp3")                      

wer_results = {}
# Iterate over each prompt file
for mp3_file in mp3_files:
    convert_to_video(root_dir+mp3_file)

In [ ]:
!python -m pip install sequence-evaluate rouge bleu transformers sentence-transformers torch rouge numpy seaborn

In [ ]:
import pandas as pd
import glob
from seq_eval import SeqEval

evaluator = SeqEval()

root_dir = "/Users/julienmiquel/dev/STT/tests/"
# Get a list of all TSV files in the current folder
tsv_files = glob.glob(root_dir=root_dir, pathname=
                      "4_short_*.tsv")
                      #"2_long*0.tsv")

file_ref = "4_short2_gemini-1.5-pro-001_0.tsv"
file_candidate = "4_short_gemini-1.5-pro-001_0.tsv"

# Read the TSV file into a dataframe
df_ref = pd.read_csv(root_dir+file_ref, sep='\t')
df_candidate = pd.read_csv(root_dir+file_candidate, sep='\t')
    
column_analyze = "text" # "speakerid" #
references = df_ref[column_analyze].to_list()
candidates = df_candidate[column_analyze].to_list()
embed_ref = embed_text(references)
embed_candidate = embed_text(candidates)




In [ ]:
scores = evaluator.evaluate(candidates, references, verbose=True)
print(scores)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim_array = cosine_similarity(X=embed_ref, Y=embed_candidate)

# display as DataFrame
axe = 'name'
# apply lambda to each row of the references list to get the first 5 letters of each string in the list
references_shorten = pd.DataFrame({'name': references})['name'].apply(lambda x: str(x)[0:100]).to_list()
candidates_shorten = pd.DataFrame({'name': candidates})['name'].apply(lambda x: str(x)[0:100]).to_list()

#df_sim = pd.DataFrame(cos_sim_array, index=references, columns=candidates)
df_sim = pd.DataFrame(cos_sim_array, index=references, columns=candidates)

print(f"columns = {len(df_sim.columns)}")
print(f"rows = {len(df_sim.index)}")

while len(df_sim.columns) < len(df_sim.values):
    id = df_sim.max(axis=1).idxmin() #.loc( df_sim.idxmax( axis='columns')) #.idxmin(axis='columns') #.apply(lambda x: np.max(x), axis='rows')
    print(f"{id}  =  {df_sim[id].max()}")
    print(f"drop id = {id}")
    df_sim = df_sim.columns.drop(id)
    print(f"columns = {len(df_sim.columns)}")
    print(f"rows = {len(df_sim.index)}")    
    

while len(df_sim.columns) > len(df_sim.values):
    id = df_sim.max(axis=0).idxmin() #.loc( df_sim.idxmax( axis='columns')) #.idxmin(axis='columns') #.apply(lambda x: np.max(x), axis='rows')
    print(f"drop id = {id}")
    df_sim.drop(index=id)
    print(f"columns = {len(df_sim.columns)}")
    print(f"rows = {len(df_sim.index)}")    

#df_sim#.idxmax( axis='columns')


#df_sim
# filter value inferior to 0.95
#df_sim.loc[ (df_sim['max'] < 0.75)]

#df_sim#['max'].idxmin( )
#df_sim['max'].filter( ,axis = 'index').idxmin()
# df_max = pd.DataFrame(df_sim.apply(lambda x: np.max(x), axis='rows'))
# df_max.columns = ["value"]
#df_max.apply(lambda x: np.min(x), axis='columns')

#df_sim['max'].min() #[df_max["value"] == df_max["max"].min()] #.apply(lambda x: np.min(x))



In [ ]:
import seaborn as sns

ax = sns.heatmap(df_sim,
                 #annot=True, 
                 cmap="crest")
#ax.xaxis.tick_top()
ax.set_xticklabels(candidates_shorten, rotation=90)
ax.set_yticklabels(references_shorten)
ax.set_title(f'Cosine Similarity for {column_analyze} (text are truncated)'  )
ax

In [ ]:
# embeddings_array = np.array(embed_ref, dtype=np.float32)
# tsne = TSNE(random_state=0, n_iter=250)
# tsne_results = tsne.fit_transform(embeddings_array)

# df_tsne = pd.DataFrame(tsne_results, columns=["TSNE1", "TSNE2"])
# df_tsne["target"] = df["target"]  # Add labels column from df_train to df_tsne

In [ ]:


from typing import List, Optional
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel


def embed_text(
    texts: List[str] ,
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "text-embedding-004",
    dimensionality: Optional[int] = 256,
) -> List[List[float]]:
    """Embeds texts with a pre-trained, foundational model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
    embeddings = model.get_embeddings(inputs, **kwargs)
    return [embedding.values for embedding in embeddings]

#embed_text(["banana muffins? ", "banana bread? banana muffins?"])

In [ ]:
whisper_file =  json.loads(open("./whisper/whisper_UGC - meeting.mp3_medium.json").read())
df_whisper = pd.DataFrame(whisper_file["segments"])
df_whisper.drop(columns=["id","seek","tokens","temperature","avg_logprob","compression_ratio","no_speech_prob"], inplace = True)

df_whisper

In [ ]:
import pandas as pd
import glob

root_dir = "/Users/julienmiquel/dev/STT/tests/"
# Get a list of all TSV files in the current folder
tsv_files = glob.glob(root_dir=root_dir, pathname=
                      "4_short_*.tsv")
                      #"2_long*0.tsv")

if len(tsv_files) > 0:
    print("Found TSV files")
    # Create an empty list to store the dataframes
    dfs = []

    # Iterate over each TSV file
    for file in tsv_files:
        # Read the TSV file into a dataframe
        df = pd.read_csv(root_dir+file, sep='\t')
        df['file'] = file
        
        # Append the dataframe to the list
        dfs.append(df)

    # Concatenate all dataframes into a single dataframe
    combined_df = pd.concat(dfs, ignore_index=True)

    # Now you have a single dataframe 'combined_df' containing data from all TSV files
    print(combined_df)

combined_df

In [ ]:
df["text"]

In [ ]:
from pydub import AudioSegment
from pydub.utils import mediainfo_json
import os
from pathlib import Path

import pandas as pd
import glob

root_dir = "/Users/julienmiquel/dev/STT/assets/"
output_dir = "/Users/julienmiquel/dev/STT/assets/resampling/"
# Get a list of all TSV files in the current folder
mp3_files = glob.glob(root_dir=root_dir, pathname=
                      "*.mp3")
                      #"2_long*0.tsv")
bitrates = [64, 92, 128, 256, 312]

# Iterate over each TSV file
for file in mp3_files:
    prompt = """Chapterize the audio content by grouping the audio content into chapters by detecting silence. 
Please only capture key events and highlights. If you are not sure about any info, please do not make it up. 
Return the result in the JSON format with keys as follows : 'timecode_start', 'timecode_stop'"""

    sound = AudioSegment.from_mp3(root_dir+file)
    print(80*"__")
    print(f"file = {file}")

    print(f"duration_seconds = {sound.duration_seconds}")
    print(f"sample_width = {sound.sample_width}")
    print(f"channels = {sound.channels}")
    print(f"frame_rate = {sound.frame_rate}")
    
    sound.export(output_dir+file+f"-{bitrate}.mp3", format="mp3", bitrate=bitrate)
    break
    

#print(f"duration = {sound.duration}")

In [ ]:
root_dir = "/Users/julienmiquel/dev/STT/assets/"
output_dir = "/Users/julienmiquel/dev/STT/assets/assets_splitted/"


def splitAudio(root_dir, file, start, stop, output_dir):
    sound = AudioSegment.from_mp3(root_dir+file)
    print(80*"__")
    print(f"file = {file}")

    sound = sound[start:stop]
    print(f"duration_seconds = {sound.duration_seconds}")
    print(f"sample_width = {sound.sample_width}")
    print(f"channels = {sound.channels}")
    print(f"frame_rate = {sound.frame_rate}")
    file_segment = output_dir+file+f"-{start}-{stop}.mp3"

    sound.export(file_segment, format="mp3")
    return file_segment

splitAudio(root_dir, "Lionel 1.mp3", 10, 15,output_dir )

In [ ]:
model_gemini = GenerativeModel(model_name)

# Iterate over each mp3 file
file = "UGC_meeting.mp3"
file = "Lionel 1.mp3"

audio1 = Part.from_uri(
    mime_type="audio/mp3",
    uri=f"gs://ml-demo-eu/datasets/input/{file}")

# prompt_split = """Chapterize the audio content by grouping the audio content into chapters by detecting silence. 
# Please only capture key events and highlights. If you are not sure about any info, please do not make it up. 
# Return the result in the JSON format with keys as follows : 'timecode_start', 'timecode_stop' 'description'
# """

prompt = """<Task>
    Transcribe in {language} this recording from the time_start 0 to the time_stop 500 of the recording.
    Identify each persons with their name and their genre if possible.
    Accuracy: Prioritize precision in capturing spoken words. Strive to minimize errors and misinterpretations.
    Clarity: Ensure the transcribed text flows naturally and is easy to understand. Punctuate sentences appropriately.

    STOP generating output properly (keep a valid JSON) after generating 8000 characters in output.
</Task>

<answer_format>
    JSON keys: id (speaker id), start (time_start), stop (time_stop), text.
    Id is numeric id of the speaker (speaker_id). Check twice the speaker id is correctly associated with the text.
    start (time_start) and stop (time_stop) are the full time in miliseconds and not truncated.
    text is the transcription of the voice of the speaker.
</answer_format>
"""

response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "time_start": {
                "type": "INTEGER"
            },
            "time_stop": {
                "type": "INTEGER"
            },
            "id": {
                "type": "STRING"
            },
            "text": {
                "type": "STRING"
            }
        },
        # "required": [
        #     "timecode_start"
        # ]
    }
}

# generation_config_json = {
#       "max_output_tokens": 8192,
#       "temperature": 0.0,
#       "top_p": 1,
#       "response_mime_type": "application/json",
#       "response_schema" : response_schema
#       #"enableStandaloneAudioTimestampsToModels": True,
      
#   }

generation_config_json = generation_config=GenerationConfig(
    max_output_tokens= 8192,
    temperature= 0.0,
    top_p= 0.0,
    frequency_penalty = 0.5,
    response_mime_type="application/json", 
    response_schema=response_schema
    )

prompts = [audio1, prompt]
response = model_gemini.generate_content(
    contents=prompts,
    generation_config=generation_config_json,
    #safety_settings=safety_settings,
    stream=False,
)

response.text

In [ ]:
import json
audio_split = json.load(response.text)
audio_split

In [ ]:
def get_audio_sequence(file):
    from pydub import AudioSegment, silence

    myaudio = AudioSegment.from_mp3(file)
    dBFS=myaudio.dBFS

    speak = silence.detect_nonsilent(myaudio, min_silence_len=500, silence_thresh=dBFS-20, seek_step=10)
    speak_sequences = speak

    #speak_sequences = [((start/1000),(stop/1000)) for start,stop in speak] #convert to sec

    print(speak_sequences)
    return speak_sequences

speak_sequences = get_audio_sequence(root_dir+file)

# filter speak_sequences when stop - start are less than 2 secondes
speak_sequences_filtered = [(start, stop) for start, stop in speak_sequences if stop - start >= 1500]

model_gemini = GenerativeModel(model_name)

prompt = """<Task>
    Transcribe in {language} this recording from the time_start 0 to the time_stop 500 of the recording.
    Identify each persons with their name and their genre if possible.
    Accuracy: Prioritize precision in capturing spoken words. Strive to minimize errors and misinterpretations.
    Clarity: Ensure the transcribed text flows naturally and is easy to understand. Punctuate sentences appropriately.

    STOP generating output properly (keep a valid JSON) after generating 8000 characters in output.
</Task>

<answer_format>
    JSON keys: id (speaker id), start (time_start), stop (time_stop), text.
    Id is numeric id of the speaker (speaker_id). Check twice the speaker id is correctly associated with the text.
    start (time_start) and stop (time_stop) are the full time in seconds and not truncated.
    text is the transcription of the voice of the speaker.
</answer_format>
"""
splited_files = []
splited_answers = []
for (start, stop) in speak_sequences_filtered:
    
    splited_file = splitAudio(root_dir,file, start, stop, output_dir)

    # Load file bytes
    with open(splited_file, "rb") as f:
        file_part = Part.from_data(data=f.read(), mime_type="audio/mp3")

    print(splited_file)
    splited_files.append(splited_file)

    # Load contents
    contents = [file_part, prompt]

    # Send to Gemini with GenerationConfig
    response = model_gemini.generate_content(contents, generation_config=generation_config_json)
    print(response.text)
    splited_answers.append(response.text)
    
    


In [ ]:
# Import the Speech-to-Text client library
from google.cloud import speech

# Instantiates a client
client = speech.SpeechClient() 

audio_content = None

# transcribe speech
audio = speech.RecognitionAudio(content=audio_content)

diarizationConfig = speech.SpeakerDiarizationConfig(enable_speaker_diarization=True)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=24000,
    language_code="fr-FR",
    model="default",
    audio_channel_count=1,
    enable_word_confidence=True,
    enable_word_time_offsets=True,
    diarization_config = diarizationConfig,
)

# Detects speech in the audio file
operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=90)

for result in response.results:
  print("Transcript: {}".format(result.alternatives[0].transcript))

In [ ]:
def transcribe_streaming(stream_file: str) -> speech.RecognitionConfig:
    """Streams transcription of the given audio file."""

    client = speech.SpeechClient()

    with open(stream_file, "rb") as audio_file:
        content = audio_file.read()

    # In practice, stream should be a generator yielding chunks of audio data.
    stream = [content]

    requests = (
        speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in stream
    )

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )

    streaming_config = speech.StreamingRecognitionConfig(config=config)

    # streaming_recognize returns a generator.
    responses = client.streaming_recognize(
        config=streaming_config,
        requests=requests,
    )

    for response in responses:
        # Once the transcription has settled, the first result will contain the
        # is_final result. The other results will be for subsequent portions of
        # the audio.
        for result in response.results:
            print(f"Finished: {result.is_final}")
            print(f"Stability: {result.stability}")
            alternatives = result.alternatives
            # The alternatives are ordered from most likely to least.
            for alternative in alternatives:
                print(f"Confidence: {alternative.confidence}")
                print(f"Transcript: {alternative.transcript}")



In [ ]:
!pip install google-cloud-speech

In [ ]:
!pip install deepgram-sdk

In [ ]:
from deepgram import DeepgramClient, PrerecordedOptions

# The API key we created in step 3
DEEPGRAM_API_KEY = 'ac4855260ef2e43fb49307f2e58e69aee84e8f96'

# Replace with your file path
PATH_TO_FILE = root_dir + "Lionel 1.mp3"

def main():
    deepgram = DeepgramClient(DEEPGRAM_API_KEY)

    with open(PATH_TO_FILE, 'rb') as buffer_data:
        payload = { 'buffer': buffer_data }

        options = PrerecordedOptions(
            punctuate=True, model="nova-2", language="fr"
        )

        response = deepgram.listen.prerecorded.v('1').transcribe_file(payload, options)
        print(response.to_json(indent=4))


main()